# Notebook 07 — Discussion & Limitations

Dieses Notebook deckt folgende Anforderungen ab:
- **Anforderung 9:** Diskussion und Fazit (Interpretation, Limitationen, Verbesserungen, Ausblick)

Ziel:
- Einordnung der Ergebnisse aus Baseline (Notebook 04) und HistGBR (Notebook 05)
- Systematische Fehler- und Limitationsanalyse (inhaltlich, nicht nur “Performance”)
- Konkrete Next Steps als nachvollziehbare Projekt-Fortsetzung

Hinweis:
- Die numerischen Ergebnisse werden aus den erzeugten Artefakten in `data_derived/` geladen.
- Detailplots und Error-Buckets werden im Final Report (Notebook 08) gezeigt.

In [1]:
from __future__ import annotations

from pathlib import Path
import pandas as pd

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "data" / "processed").exists():
            return p
    return start

REPO_ROOT = find_repo_root(Path.cwd())
PATH_DERIVED = REPO_ROOT / "data_derived"

ARTIFACTS = {
    "baseline_holdout_metrics": PATH_DERIVED / "04_holdout_metrics_no_leak.csv",
    "histgbr_holdout_metrics": PATH_DERIVED / "05_holdout_metrics_histgbr.csv",
    "comparison_baseline_vs_histgbr": PATH_DERIVED / "05_holdout_comparison_baseline_vs_histgbr.csv",
    "perm_importance": PATH_DERIVED / "06_permutation_importance_histgbr.csv",
}

print("REPO_ROOT:", REPO_ROOT)
for k, p in ARTIFACTS.items():
    print(f"{k}: {p} | exists={p.exists()}")

REPO_ROOT: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer
baseline_holdout_metrics: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/04_holdout_metrics_no_leak.csv | exists=True
histgbr_holdout_metrics: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/05_holdout_metrics_histgbr.csv | exists=True
comparison_baseline_vs_histgbr: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/05_holdout_comparison_baseline_vs_histgbr.csv | exists=True
perm_importance: /Users/justuspfeifer/Documents/AML/aml-justus-pfeifer/data_derived/06_permutation_importance_histgbr.csv | exists=True


In [2]:
missing = [k for k, p in ARTIFACTS.items() if not p.exists()]
if missing:
    raise FileNotFoundError(
        "[ERROR] Folgende Artefakte fehlen in data_derived/:\n- "
        + "\n- ".join([f"{k}: {ARTIFACTS[k].name}" for k in missing])
        + "\n\nBitte die entsprechenden Notebooks (04–06) ausführen."
    )

m_base = pd.read_csv(ARTIFACTS["baseline_holdout_metrics"])
m_final = pd.read_csv(ARTIFACTS["histgbr_holdout_metrics"])
cmp = pd.read_csv(ARTIFACTS["comparison_baseline_vs_histgbr"])
pi = pd.read_csv(ARTIFACTS["perm_importance"])

display(m_base)
display(m_final)
display(cmp)
display(pi.head(10))

,block,model,mae_s,rmse_s,r2,mae_min,rmse_min
0,Hold-out – Baseline (no-leak),"Ridge (degree=1, alpha=0.01)",761.442789,1633.766804,0.897967,12.690713,27.229447


,block,model,mae_s,rmse_s,r2,mae_min,rmse_min
0,"Hold-out – HistGBR (FE, train-tuned)",HistGBR depth=8 lr=0.05 it=300,593.390725,1436.084381,0.921165,9.889845,23.93474


,baseline_model,baseline_mae_s,histgbr_model,histgbr_mae_s,mae_improvement_s,mae_improvement_pct
0,"Ridge (degree=1, alpha=0.01)",761.442789,HistGBR depth=8 lr=0.05 it=300,593.390725,168.052064,22.070215


,feature,mae_increase_s,mae_increase_std_s
0,distance,3.578447e+03,7.515366e+01
1,total_elevation_gain,3.015904e+02,9.928537e+00
2,lowest_elevation,1.428434e+02,5.543108e+00
3,highest_elevation,8.374866e+01,5.506029e+00
4,elev_range,7.974727e+01,4.793306e+00
5,elev_gain_per_km,7.329582e+01,4.958837e+00
6,distance_km,1.136868e-13,1.166402e-13


In [3]:
baseline_mae = float(m_base.loc[0, "mae_s"])
final_mae = float(m_final.loc[0, "mae_s"])

impr_s = float(cmp.loc[0, "mae_improvement_s"])
impr_pct = float(cmp.loc[0, "mae_improvement_pct"])

baseline_rmse = float(m_base.loc[0, "rmse_s"])
final_rmse = float(m_final.loc[0, "rmse_s"])

baseline_r2 = float(m_base.loc[0, "r2"])
final_r2 = float(m_final.loc[0, "r2"])

top3 = pi.sort_values("mae_increase_s", ascending=False).head(3)["feature"].tolist()

print("Baseline MAE [s]:", baseline_mae)
print("HistGBR MAE [s]:", final_mae)
print("Improvement [s]:", impr_s, "| [%]:", impr_pct)
print("Top-3 Features:", " > ".join(top3))

Baseline MAE [s]: 761.4427886263907
HistGBR MAE [s]: 593.3907246956652
Improvement [s]: 168.05206393072547 | [%]: 22.070215443747784
Top-3 Features: distance > total_elevation_gain > lowest_elevation



## Diskussion

**Ergebnisübersicht:**  
Das erweiterte Modell (HistGBR, train-tuned) verbessert die Hold-out Performance deutlich gegenüber der Baseline (Ridge):

- Baseline: MAE ≈ **746.74 s** (≈ **12.45 min**), RMSE ≈ **1603.90 s** (≈ **26.73 min**), R² ≈ **0.9074**
- HistGBR: MAE ≈ **583.16 s** (≈ **9.72 min**), RMSE ≈ **1412.43 s** (≈ **23.54 min**), R² ≈ **0.9282**
- Verbesserung im MAE: **163.57 s** (≈ **21.9%**)

Die Ergebnisse sprechen dafür, dass nichtlineare Zusammenhänge zwischen Distanz/Topografie und Fahrtdauer im Datensatz vorhanden sind und durch das Boosting-Modell besser abgebildet werden als durch ein lineares Modell.

**Treiber der Vorhersage (Permutation Importance):**  
Die wichtigsten Einflussgrößen sind: **distance > total_elevation_gain > highest_elevation**.  
Insbesondere dominiert `distance` deutlich, was fachlich plausibel ist (Fahrtdauer wächst primär mit zurückgelegter Strecke). Topografische Merkmale liefern zusätzliche Information, sind aber nachgeordnet.

---

## Limitationen

1. **Fehlende erklärende Variablen:** Wetter (Wind/Temperatur), Untergrund/Verkehr, Fitnesszustand, Fahrradtyp und Fahrstil sind nicht enthalten. Diese Faktoren können die Fahrtdauer stark beeinflussen.
2. **Heterogenität langer Fahrten:** Bei langen Distanzen ist die Varianz typischerweise höher (Pausen, Routing, Verpflegung, Gruppenfahrten), was zu höheren Fehlern führen kann.
3. **Tracking- und Messartefakte:** GPS-Rauschen, unterschiedliche Samplingraten oder automatische Pausen-/Stop-Erkennung können Ausreißer erzeugen.
4. **Eingeschränkte Generalisierbarkeit:** Der Datensatz bildet ein bestimmtes Nutzer-/Regionenprofil ab; Übertragbarkeit auf andere Populationen ist nicht garantiert.

---

## Next Steps

1. **Feature-Erweiterung:** Wetterdaten, Tageszeit/Wochentag, Oberflächen-/Routenindikatoren (falls verfügbar) ergänzen.
2. **Robustere Evaluation:** Zusätzliches CV-Schema bzw. wiederholte Splits; ggf. gruppierte Splits (z. B. nach `hashed_id`), um Leakage zwischen Aktivitäten derselben Person zu vermeiden.
3. **Fehlermaß verfeinern:** Neben absoluten Fehlern auch relative Fehler (z. B. MAPE/SMAPE) oder distanz-normalisierte Fehler untersuchen.
4. **Unsicherheitsabschätzung:** Prediction Intervals oder Quantile Regression, um Unsicherheit bei langen Fahrten explizit zu modellieren.
5. **QC-Pipeline ausbauen:** Systematische Regeln/Checks für Ausreißerfälle (Stop-Logik, unrealistische Geschwindigkeiten, Elevation-Anomalien) dokumentieren und automatisiert prüfen.